# Regression Week 1: Simple Linear Regression

In this notebook we will use data on house sales in King County to predict house prices using simple (one input) linear regression. You will:
* Use graphlab SArray and SFrame functions to compute important summary statistics
* Write a function to compute the Simple Linear Regression weights using the closed form solution
* Write a function to make predictions of the output given the input feature
* Turn the regression around to predict the input given the output
* Compare two different models for predicting house prices

In this notebook you will be provided with some already complete code as well as some code that you should complete yourself in order to answer quiz questions. The code we provide to complte is optional and is there to assist you with solving the problems but feel free to ignore the helper code and write your own.

# Fire up graphlab create

In [1]:
import graphlab

# Load house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [23]:
sales = graphlab.SFrame('kc_house_data.gl/')

# Split data into training and testing

We use seed=0 so that everyone running this notebook gets the same results.  In practice, you may set a random seed (or let GraphLab Create pick a random seed for you).  

In [24]:
train_data,test_data = sales.random_split(.8,seed=0)

# Useful SFrame summary functions

In order to make use of the closed form soltion as well as take advantage of graphlab's built in functions we will review some important ones. In particular:
* Computing the sum of an SArray
* Computing the arithmetic average (mean) of an SArray
* multiplying SArrays by constants
* multiplying SArrays by other SArrays

In [6]:
# Let's compute the mean of the House Prices in King County in 2 different ways.
prices = sales['price'] # extract the price column of the sales SFrame -- this is now an SArray

# recall that the arithmetic average (the mean) is the sum of the prices divided by the total number of houses:
sum_prices = prices.sum()
num_houses = prices.size() # when prices is an SArray .size() returns its length
avg_price_1 = sum_prices/num_houses
avg_price_2 = prices.mean() # if you just want the average, the .mean() function
print "average price via method 1: " + str(avg_price_1)
print "average price via method 2: " + str(avg_price_2)

average price via method 1: 540088.141905
average price via method 2: 540088.141905


As we see we get the same answer both ways

In [8]:
# if we want to multiply every price by 0.5 it's a simple as:
half_prices = 0.5*prices
# Let's compute the sum of squares of price. We can multiply two SArrays of the same length elementwise also with *
prices_squared = prices*prices
print "the prices squares are: "+str(prices_squared)
sum_prices_squared = prices_squared.sum() # price_squared is an SArray of the squares and we want to add them up.
print "the sum of price squared is: " + str(sum_prices_squared)

the prices squares are: [49239610000.0, 289444000000.0, 32400000000.0, 364816000000.0, 260100000000.0, 1500625000000.0, 66306250000.0, 85176422500.0, 52670250000.0, 104329000000.0, 438906250000.0, 219024000000.0, 96100000000.0, 160000000000.0, 280900000000.0, 422500000000.0, 156025000000.0, 235225000000.0, 35721000000.0, 52900000000.0, 148225000000.0, 4000000000000.0, 81225000000.0, 63857290000.0, 108241000000.0, 54289000000.0, 877969000000.0, 444889000000.0, 191844000000.0, 516961000000.0, 336980250000.0, 78400000000.0, 472656250000.0, 286225000000.0, 104006250000.0, 484416000000.0, 302500000000.0, 409600000000.0, 57600000000.0, 366025000000.0, 390625000000.0, 600625000000.0, 743026760100.0, 469225000000.0, 95481000000.0, 238144000000.0, 44306040100.0, 616225000000.0, 202500000000.0, 1822500000000.0, 51984000000.0, 119025000000.0, 360000000000.0, 342225000000.0, 846400000000.0, 783225000000.0, 85556250000.0, 90601000000.0, 904401000000.0, 184900000000.0, 422500000000.0, 83521000000.0,

Aside: The python notation x.xxe+yy means x.xx \* 10^(yy). e.g 100 = 10^2 = 1*10^2 = 1e2 

# Build a generic simple linear regression function 

Armed with these SArray functions we can use the closed form solution found from lecture to compute the slope and intercept for a simple linear regression on observations stored as SArrays: input_feature, output.

Complete the following function (or write your own) to compute the simple linear regression slope and intercept:

In [2]:
def simple_linear_regression(input_feature, output):
    
    # compute the mean of  input_feature and output
    #mean is the avg
    mean_input_feature   = input_feature.mean()
    mean_output          = output.mean()  
    
    # compute the product of the output and the input_feature and its mean
    product_input_output         = output*input_feature
    mean_product_input_output    = product_input_output.mean()
    
    # compute the squared value of the input_feature and its mean
    each_input_squared_mean      = input_feature.mean()**2
    input_feature_squared        = input_feature*input_feature
    input_feature_squared_mean   = input_feature_squared.mean()
    
    
    # use the formula for the slope
    #m= (avg. of x * avg of y )-(product of avg of x and y)/((avg of x)^2 - (each x squared with avg. )
    
    slope = ((mean_output*mean_input_feature)-mean_product_input_output)/(each_input_squared_mean-input_feature_squared_mean)
    
    # use the formula for the intercept
    #f(x)=W0+W1*x
    #W1=slope
    #W0=intercept
    
    #W0 = b = avg of y - slope * avg of x
    intercept = mean_output - (slope* mean_input_feature)
    
    return (intercept, slope)

We can test that our function works by passing it something where we know the answer. In particular we can generate a feature and then put the output exactly on a line: output = 1 + 1\*input_feature then we know both our slope and intercept should be 1

In [9]:
test_feature = graphlab.SArray(range(5))
print test_feature
test_output = graphlab.SArray(1 + 1*test_feature)
print test_output
(test_intercept, test_slope) =  simple_linear_regression(test_feature, test_output)
print "Intercept: " + str(test_intercept)
print "Slope: " + str(test_slope)

[0L, 1L, 2L, 3L, 4L]
[1L, 2L, 3L, 4L, 5L]
Intercept: 1.0
Slope: 1.0


## How to create an SArray

In [11]:
test_feature_2 = graphlab.SArray(data=[1,2,3,4,5], dtype=int)
test_output_2 = graphlab.SArray(data=[1,2,3,4,5], dtype=int)
print test_feature_2
print test_output_2

[1L, 2L, 3L, 4L, 5L]
[1L, 2L, 3L, 4L, 5L]


Now that we know it works let's build a regression model for predicting price based on sqft_living. Rembember that we train on train_data!

In [25]:
sqft_intercept, sqft_slope = simple_linear_regression(train_data['sqft_living'], train_data['price'])

print "Intercept: " + str(sqft_intercept)
print "Slope: " + str(sqft_slope)

Intercept: -47116.0765749
Slope: 281.958838568


### The price of the house increases by 282 US dollars /sq.ft

# Predicting Values

Now that we have the model parameters: intercept & slope we can make predictions. Using SArrays it's easy to multiply an SArray by a constant and add a constant value. Complete the following function to return the predicted output given the input_feature, slope and intercept:

In [28]:
def get_regression_predictions(input_feature, intercept, slope):
    # calculate the predicted values:
    #f(x) = intercept + slope * x
    
    predicted_values = intercept + slope*input_feature
    
    return predicted_values

Now that we can calculate a prediction given the slop and intercept let's make a prediction. Use (or alter) the following to find out the estimated price for a house with 2650 squarefeet according to the squarefeet model we estiamted above.

**Quiz Question: Using your Slope and Intercept from (4), What is the predicted price for a house with 2650 sqft?**

In [29]:
my_house_sqft = 2650
estimated_price = get_regression_predictions(my_house_sqft, sqft_intercept, sqft_slope)
print "The estimated price for a house with %d squarefeet is $%.2f" % (my_house_sqft, estimated_price)

The estimated price for a house with 2650 squarefeet is $700074.85


# Residual Sum of Squares

Now that we have a model and can make predictions let's evaluate our model using Residual Sum of Squares (RSS). Recall that RSS is the sum of the squares of the residuals and the residuals is just a fancy word for the difference between the predicted output and the true output. 

Complete the following (or write your own) function to compute the RSS of a simple linear regression model given the input_feature, output, intercept and slope:

In [54]:
def get_residual_sum_of_squares(input_feature, output, intercept, slope):
    
    RSS = 0
    
    #The size of both SArrays must the same.
    if len(output)==len(input_feature):
        
        for i in range(len(output)):
            RSS = RSS + (output[i] - (intercept+slope*input_feature[i]))**2
    else: 
        
        print "The SArrays do not match the sizes. Verify your input values."
        RSS=0
               
    
    # then compute the residuals (since we are squaring it doesn't matter which order you subtract)

    # square the residuals and add them up

    return(RSS)

Let's test our get_residual_sum_of_squares function by applying it to the test model where the data lie exactly on a line. Since they lie exactly on a line the residual sum of squares should be zero!

In [55]:
print get_residual_sum_of_squares(test_feature, test_output, test_intercept, test_slope) # should be 0.0

0.0


Now use your function to calculate the RSS on training data from the squarefeet model calculated above.

**Quiz Question: According to this function and the slope and intercept from the squarefeet model What is the RSS for the simple linear regression using squarefeet to predict prices on TRAINING data?**

In [56]:
rss_prices_on_sqft = get_residual_sum_of_squares(train_data['sqft_living'], train_data['price'], sqft_intercept, sqft_slope)
print 'The RSS of predicting Prices based on Square Feet is : ' + str(rss_prices_on_sqft)

The RSS of predicting Prices based on Square Feet is : 1.20191835632e+15


# Predict the squarefeet given price

What if we want to predict the squarefoot given the price? Since we have an equation y = a + b\*x we can solve the function for x. So that if we have the intercept (a) and the slope (b) and the price (y) we can solve for the estimated squarefeet (x).

Comlplete the following function to compute the inverse regression estimate, i.e. predict the input_feature given the output!

In [57]:
def inverse_regression_predictions(output, intercept, slope):
    #y=mx+b solve for x
    #x=(y-b)/m
    
    # solve output = intercept + slope*input_feature for input_feature. 
    #Use this equation to compute the inverse predictions:
    estimated_feature = (output - intercept)/slope
    return estimated_feature

Now that we have a function to compute the squarefeet given the price from our simple regression model let's see how big we might expect a house that coses $800,000 to be.

**Quiz Question: According to this function and the regression slope and intercept from (3) what is the estimated square-feet for a house costing $800,000?**

In [58]:
my_house_price = 800000
estimated_squarefeet = inverse_regression_predictions(my_house_price, sqft_intercept, sqft_slope)
print "The estimated squarefeet for a house worth $%.2f is %d" % (my_house_price, estimated_squarefeet)

The estimated squarefeet for a house worth $800000.00 is 3004


In [72]:
houses_w_sqft_3000 = sales[sales['sqft_living']<3005].sort('sqft_living',ascending=False)

# New Model: estimate prices from bedrooms

In [75]:
houses_w_sqft_3000.sort('sqft_living',ascending=False)

id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront
8944460290,2014-10-23 00:00:00+00:00,398000.0,5.0,2.5,3004.0,5700,2,0
7936800150,2014-07-02 00:00:00+00:00,394500.0,4.0,2.5,3002.0,6042,2,0
2122059206,2014-05-13 00:00:00+00:00,373000.0,5.0,2.5,3001.0,5710,2,0
4305600250,2014-10-27 00:00:00+00:00,540000.0,4.0,2.5,3000.0,5471,2,0
0726059048,2014-09-26 00:00:00+00:00,490500.0,3.0,2.0,3000.0,21883,2,0
3574801500,2014-09-26 00:00:00+00:00,490000.0,4.0,2.5,3000.0,8645,2,0
2025079033,2014-12-10 00:00:00+00:00,415000.0,1.0,2.0,3000.0,204732,2.5,0
0624100950,2015-03-11 00:00:00+00:00,850000.0,3.0,2.25,3000.0,18450,1,0
2324800070,2014-06-10 00:00:00+00:00,740000.0,3.0,2.5,3000.0,25341,2,0
3425059076,2014-09-22 00:00:00+00:00,780000.0,2.0,3.25,3000.0,24004,1,0


We have made one model for predicting house prices using squarefeet, but there are many other features in the sales SFrame. 
Use your simple linear regression function to estimate the regression parameters from predicting Prices based on number of bedrooms. Use the training data!

In [76]:
# Estimate the slope and intercept for predicting 'price' based on 'bedrooms'

bedroom_intercept, bedroom_slope = simple_linear_regression(train_data['bedrooms'], train_data['price'])

print "Intercept: " + str(bedroom_intercept)
print "Slope: " + str(bedroom_slope)

Intercept: 109473.180469
Slope: 127588.952175


# Test your Linear Regression Algorithm

Now we have two models for predicting the price of a house. How do we know which one is better? Calculate the RSS on the TEST data (remember this data wasn't involved in learning the model). Compute the RSS from predicting prices using bedrooms and from predicting prices using squarefeet.

**Quiz Question: Which model (square feet or bedrooms) has lowest RSS on TEST data? Think about why this might be the case.**

# Compute RSS when using bedrooms on TEST data:

rss_prices_on_bedrooms = get_residual_sum_of_squares(test_data['bedrooms'], test_data['price'], bedroom_intercept, bedroom_slope)
print 'The RSS of predicting Prices based on bedrooms is : ' + str(rss_prices_on_bedrooms)

In [78]:
# Compute RSS when using squarfeet on TEST data:

rss_prices_on_sqft = get_residual_sum_of_squares(test_data['sqft_living'], train_data['price'], sqft_intercept, sqft_slope)
print 'The RSS of predicting Prices based on Square Feet is : ' + str(rss_prices_on_sqft)

The RSS of predicting Prices based on Square Feet is : 2.75402936247e+14
